In [ ]:
pip install mysql-connector-python

In [ ]:
import mysql.connector
from mysql.connector import Error

hostname = "29x98l.h.filess.io"
database = "ecommercedataset_scientist"
port = "3307"
username = "ecommercedataset_scientist"
password = "3920ec9104b5d9d1ddeb3d44ef3fc0af3a199935"

try:
    connection = mysql.connector.connect(host=hostname, database=database, user=username, password=password, port=port)
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")



/tmp/ipython-input-6-2058088896.py:13: DeprecationWarning: Call to deprecated function get_server_info. Reason: 
    The property counterpart 'server_info' should be used instead.

  db_Info = connection.get_server_info()


Connected to MySQL Server version  8.0.36-28
You're connected to database:  ('ecommercedataset_scientist',)
MySQL connection is closed


In [ ]:
import pandas as pd

order_payments = pd.read_csv("olist_order_payments_dataset.csv")
order_payments.head()


,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [ ]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

#connection details
hostname = "29x98l.h.filess.io"
database = "ecommercedataset_scientist"
port = "3307"
username = "ecommercedataset_scientist"
password = "3920ec9104b5d9d1ddeb3d44ef3fc0af3a199935"

#csv file path
csv_file_path = "olist_order_payments_dataset.csv"

#table name where the data will be uploaded
table_name = "olist_order_payments"

try:
    #step1: establish a connection to mysql server
    connection = mysql.connector.connect(
        host=hostname,
        database=database,
        user=username,
        password=password,
        port=port)

    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)

        #step2: create a cursor to execute sql queries
        cursor = connection.cursor()

        #step3: drop table if it already exists
        cursor.execute(f"DROP TABLE IF EXISTS {table_name};")
        print(f"Table `{table_name}` dropped if it existed.")

        #step4: create a table structure to match csv file
        create_table_query = f"""
        CREATE TABLE {table_name} (
            order_id VARCHAR(255),
            payment_sequential INT,
            payment_type VARCHAR(255),
            payment_installments INT,
            payment_value FLOAT
        );
        """
        cursor.execute(create_table_query)
        print(f"Table `{table_name}` created successfully.")

        #step5: load the csv data into pandas data frame
        data = pd.read_csv(csv_file_path)

        #step6: insert data in batches of 1000 records
        batch_size = 1000
        total_records = len(data)

        print(f"Inserting data in batches of {batch_size} records...")
        for start in range(0, total_records, batch_size):
            end = start + batch_size
            batch = data.iloc[start:end] # get the current batch of records

            #convert batch to list of tuples for mysql insertion
            batch_records = [
                tuple(row) for row in batch.itertuples(index=False, name=None)
            ]

            #prepare the insert query
            insert_query = f"""
            INSERT INTO {table_name}
            (order_id, payment_sequential, payment_type, payment_installments, payment_value)
            VALUES (%s, %s, %s, %s, %s);
            """

            #execute the insertion query for the batch
            cursor.executemany(insert_query, batch_records)
            connection.commit() # commit after each batch
            print(f"Inserted records {start+1} to {min(end, total_records)} successfully.")

        print(f"All {total_records} records inserted succesfully into `{table_name}`.")

except Error as e:
    #step7: handle any errors
    print("Error while connecting to MySQL or inserting data: ", e)
finally:
    #step8: close the cursor and connection
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")


/tmp/ipython-input-8-402468952.py:28: DeprecationWarning: Call to deprecated function get_server_info. Reason: 
    The property counterpart 'server_info' should be used instead.

  db_Info = connection.get_server_info()


Connected to MySQL Server version  8.0.36-28
Table `olist_order_payments` dropped if it existed.
Table `olist_order_payments` created successfully.
Inserting data in batches of 1000 records...
Inserted records 1 to 1000 successfully.
Inserted records 1001 to 2000 successfully.
Inserted records 2001 to 3000 successfully.
Inserted records 3001 to 4000 successfully.
Inserted records 4001 to 5000 successfully.
Inserted records 5001 to 6000 successfully.
Inserted records 6001 to 7000 successfully.
Inserted records 7001 to 8000 successfully.
Inserted records 8001 to 9000 successfully.
Inserted records 9001 to 10000 successfully.
Inserted records 10001 to 11000 successfully.
Inserted records 11001 to 12000 successfully.
Inserted records 12001 to 13000 successfully.
Inserted records 13001 to 14000 successfully.
Inserted records 14001 to 15000 successfully.
Inserted records 15001 to 16000 successfully.
Inserted records 16001 to 17000 successfully.
Inserted records 17001 to 18000 successfully.
I

In [1]:
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 20.9 MB/s eta 0:00:00


In [4]:
# importing module
from pymongo import MongoClient

hostname = "0kwesc.h.filess.io"
database = "ecommercedatasetnosql_helloturn"
port = "61004"
username = "ecommercedatasetnosql_helloturn"
password = "e9ad5339e1b3e00ef4a9c2ab1074f2bdff77f63b"

uri = "mongodb://" + username + ":" + password + "@" + hostname + ":" + port + "/" + database

# Connect with the portnumber and host
client = MongoClient(uri)

# Access database
mydatabase = client[database]


read the product_category csv and create a colection and upload it to above mongodb

In [ ]:
import pandas as pd
from pymongo import MongoClient

# Read the product_category_name_translation.csv file into a pandas DataFrame
try:
    product_categories = pd.read_csv("product_category_name_translation.csv")
    print("CSV file read successfully.")
except FileNotFoundError:
    print("Error: 'product_category_name_translation.csv' not found.")
    product_categories = None # Set to None to prevent further execution if file not found

if product_categories is not None:
    # Access the MongoDB database using the existing client
    mydatabase = client[database]

    # Define the collection name
    collection_name = "product_categories"

    # Drop the collection if it already exists
    if collection_name in mydatabase.list_collection_names():
        mydatabase.drop_collection(collection_name)
        print(f"Collection `{collection_name}` dropped if it existed.")

    # Create a new collection
    mycollection = mydatabase[collection_name]
    print(f"Collection `{collection_name}` created.")

    # Convert the DataFrame to a list of dictionaries
    data_dict = product_categories.to_dict(orient='records')

    # Insert the data into the collection
    if data_dict:
        try:
            mycollection.insert_many(data_dict)
            print(f"Inserted {len(data_dict)} records into `{collection_name}`.")
        except Exception as e:
            print(f"Error inserting data into MongoDB: {e}")
    else:
        print("No data to insert.")

In [5]:
import pandas as pd
from pymongo import MongoClient

# Read the product_category_name_translation.csv file into a pandas DataFrame
try:
    product_categories = pd.read_csv("product_category_name_translation.csv")
    print("CSV file read successfully.")
except FileNotFoundError:
    print("Error: 'product_category_name_translation.csv' not found.")
    product_categories = None # Set to None to prevent further execution if file not found

if product_categories is not None:
    # Access the MongoDB database using the existing client
    mydatabase = client[database]

    # Define the collection name
    collection_name = "product_categories"

    # Drop the collection if it already exists
    if collection_name in mydatabase.list_collection_names():
        mydatabase.drop_collection(collection_name)
        print(f"Collection `{collection_name}` dropped if it existed.")

    # Create a new collection
    mycollection = mydatabase[collection_name]
    print(f"Collection `{collection_name}` created.")

    # Convert the DataFrame to a list of dictionaries
    data_dict = product_categories.to_dict(orient='records')

    # Insert the data into the collection
    if data_dict:
        try:
            mycollection.insert_many(data_dict)
            print(f"Inserted {len(data_dict)} records into `{collection_name}`.")
        except Exception as e:
            print(f"Error inserting data into MongoDB: {e}")
    else:
        print("No data to insert.")

CSV file read successfully.
Collection `product_categories` created.
Inserted 71 records into `product_categories`.
